# Colab-SRFlow
Original: [andreas128/SRFlow](https://github.com/andreas128/SRFlow)

My fork: [styler00dollar/Colab-SRFlow](https://github.com/styler00dollar/Colab-SRFlow)

In [ ]:
!nvidia-smi

In [ ]:
#@title install and download models
#!git clone https://github.com/andreas128/SRFlow.git
!git clone https://github.com/styler00dollar/Colab-SRFlow
%cd /content/Colab-SRFlow
!wget  http://data.vision.ee.ethz.ch/alugmayr/SRFlow/pretrained_models.zip
!unzip pretrained_models.zip
!rm pretrained_models.zip
!mkdir /content/hr/
!mkdir /content/lr/
!pip install lpips

Place files inside ```/content/hr/``` and ```/content/lr/```. (Warning: Must be same file format.)

Official code only works if HR is supplied because of metrics calculation.

In [ ]:
#@title test.py (disable HR usage / metrics)
%%writefile /content/Colab-SRFlow/code/test.py
# Copyright (c) 2020 Huawei Technologies Co., Ltd.
# Licensed under CC BY-NC-SA 4.0 (Attribution-NonCommercial-ShareAlike 4.0 International) (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode
#
# The code is released for academic research use only. For commercial use, please contact Huawei Technologies Co., Ltd.
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# This file contains content licensed by https://github.com/xinntao/BasicSR/blob/master/LICENSE/LICENSE


import glob
import sys
from collections import OrderedDict

from natsort import natsort

import options.options as option
from Measure import Measure, psnr
from imresize import imresize
from models import create_model
import torch
from utils.util import opt_get
import numpy as np
import pandas as pd
import os
import cv2


def fiFindByWildcard(wildcard):
    return natsort.natsorted(glob.glob(wildcard, recursive=True))


def load_model(conf_path):
    opt = option.parse(conf_path, is_train=False)
    opt['gpu_ids'] = None
    opt = option.dict_to_nonedict(opt)
    model = create_model(opt)

    model_path = opt_get(opt, ['model_path'], None)
    model.load_network(load_path=model_path, network=model.netG)
    return model, opt


def predict(model, lr):
    model.feed_data({"LQ": t(lr)}, need_GT=False)
    model.test()
    visuals = model.get_current_visuals(need_GT=False)
    return visuals.get('rlt', visuals.get("SR"))


def t(array): return torch.Tensor(np.expand_dims(array.transpose([2, 0, 1]), axis=0).astype(np.float32)) / 255


def rgb(t): return (
        np.clip((t[0] if len(t.shape) == 4 else t).detach().cpu().numpy().transpose([1, 2, 0]), 0, 1) * 255).astype(
    np.uint8)


def imread(path):
    return cv2.imread(path)[:, :, [2, 1, 0]]


def imwrite(path, img):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    cv2.imwrite(path, img[:, :, [2, 1, 0]])


def imCropCenter(img, size):
    h, w, c = img.shape

    h_start = max(h // 2 - size // 2, 0)
    h_end = min(h_start + size, h)

    w_start = max(w // 2 - size // 2, 0)
    w_end = min(w_start + size, w)

    return img[h_start:h_end, w_start:w_end]


def impad(img, top=0, bottom=0, left=0, right=0, color=255):
    return np.pad(img, [(top, bottom), (left, right), (0, 0)], 'reflect')


def main():
    conf_path = sys.argv[1]
    conf = conf_path.split('/')[-1].replace('.yml', '')
    model, opt = load_model(conf_path)

    lr_dir = opt['dataroot_LR']
    hr_dir = opt['dataroot_GT']

    lr_paths = fiFindByWildcard(os.path.join(lr_dir, '*.png'))
    hr_paths = fiFindByWildcard(os.path.join(hr_dir, '*.png'))

    this_dir = os.path.dirname(os.path.realpath(__file__))
    test_dir = os.path.join(this_dir, '..', 'results', conf)
    print(f"Out dir: {test_dir}")

    measure = Measure(use_gpu=False)

    fname = f'measure_full.csv'
    fname_tmp = fname + "_"
    path_out_measures = os.path.join(test_dir, fname_tmp)
    path_out_measures_final = os.path.join(test_dir, fname)

    if os.path.isfile(path_out_measures_final):
        df = pd.read_csv(path_out_measures_final)
    elif os.path.isfile(path_out_measures):
        df = pd.read_csv(path_out_measures)
    else:
        df = None

    scale = opt['scale']

    pad_factor = 2

    for lr_path, idx_test in zip(lr_paths, range(len(lr_paths))):

        lr = imread(lr_path)
        #hr = imread(hr_path)

        # Pad image to be % 2
        h, w, c = lr.shape
        lq_orig = lr.copy()
        lr = impad(lr, bottom=int(np.ceil(h / pad_factor) * pad_factor - h),
                   right=int(np.ceil(w / pad_factor) * pad_factor - w))

        lr_t = t(lr)

        heat = opt['heat']

        if df is not None and len(df[(df['heat'] == heat) & (df['name'] == idx_test)]) == 1:
            continue

        sr_t = model.get_sr(lq=lr_t, heat=heat)

        sr = rgb(torch.clamp(sr_t, 0, 1))
        sr = sr[:h * scale, :w * scale]

        path_out_sr = os.path.join(test_dir, "{:0.2f}".format(heat).replace('.', ''), "{:06d}.png".format(idx_test))
        imwrite(path_out_sr, sr)

        meas = OrderedDict(conf=conf, heat=heat, name=idx_test)
        #meas['PSNR'], meas['SSIM'], meas['LPIPS'] = measure.measure(sr, hr)

        lr_reconstruct_rgb = imresize(sr, 1 / opt['scale'])
        meas['LRC PSNR'] = psnr(lq_orig, lr_reconstruct_rgb)

        str_out = format_measurements(meas)
        #print(str_out)

        df = pd.DataFrame([meas]) if df is None else pd.concat([pd.DataFrame([meas]), df])

        df.to_csv(path_out_measures + "_", index=False)
        os.rename(path_out_measures + "_", path_out_measures)

    #df.to_csv(path_out_measures, index=False)
    #os.rename(path_out_measures, path_out_measures_final)

    #str_out = format_measurements(df.mean())
    #print(f"Results in: {path_out_measures_final}")
    #print('Mean: ' + str_out)


def format_measurements(meas):
    s_out = []
    for k, v in meas.items():
        v = f"{v:0.2f}" if isinstance(v, float) else v
        s_out.append(f"{k}: {v}")
    str_out = ", ".join(s_out)
    return str_out


if __name__ == "__main__":
    main()

In [ ]:
#@title example config (based on SRFlow_DF2K_4X)
%%writefile /content/Colab-SRFlow/code/confs/test.yml
# Copyright (c) 2020 Huawei Technologies Co., Ltd.
# Licensed under CC BY-NC-SA 4.0 (Attribution-NonCommercial-ShareAlike 4.0 International) (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode
#
# The code is released for academic research use only. For commercial use, please contact Huawei Technologies Co., Ltd.
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# This file contains content licensed by https://github.com/xinntao/BasicSR/blob/master/LICENSE/LICENSE

#### general settings
name: train
use_tb_logger: true
model: SRFlow
distortion: sr
scale: 4
gpu_ids: [ 0 ]

#### datasets
datasets:
  train:
    name: CelebA_160_tr
    mode: LRHR_PKL
    dataroot_GT: ../datasets/DF2K-tr.pklv4
    dataroot_LQ: ../datasets/DF2K-tr_X4.pklv4
    quant: 32

    use_shuffle: true
    n_workers: 3  # per GPU
    batch_size: 12
    GT_size: 160
    use_flip: true
    color: RGB
  val:
    name: CelebA_160_va
    mode: LRHR_PKL
    dataroot_GT: ../datasets/DIV2K-va.pklv4
    dataroot_LQ: ../datasets/DIV2K-va_X4.pklv4
    quant: 32
    n_max: 20

#### Test Settings
dataroot_GT: /content/hr/
dataroot_LR: /content/lr/
model_path: /content/Colab-SRFlow/pretrained_models/SRFlow_DF2K_4X.pth
heat: 0.9 # This is the standard deviation of the latent vectors

#### network structures
network_G:
  which_model_G: SRFlowNet
  in_nc: 3
  out_nc: 3
  nf: 64
  nb: 23
  upscale: 4
  train_RRDB: false
  train_RRDB_delay: 0.5

  flow:
    K: 16
    L: 3
    noInitialInj: true
    coupling: CondAffineSeparatedAndCond
    additionalFlowNoAffine: 2
    split:
      enable: true
    fea_up0: true
    stackRRDB:
      blocks: [ 1, 8, 15, 22 ]
      concat: true

#### path
path:
  pretrain_model_G: ../pretrained_models/RRDB_DF2K_4X.pth
  strict_load: true
  resume_state: auto

#### training settings: learning rate scheme, loss
train:
  manual_seed: 10
  lr_G: !!float 2.5e-4
  weight_decay_G: 0
  beta1: 0.9
  beta2: 0.99
  lr_scheme: MultiStepLR
  warmup_iter: -1  # no warm up
  lr_steps_rel: [ 0.5, 0.75, 0.9, 0.95 ]
  lr_gamma: 0.5

  niter: 200000
  val_freq: 40000

#### validation settings
val:
  heats: [ 0.0, 0.5, 0.75, 1.0 ]
  n_sample: 3

#### logger
logger:
  print_freq: 100
  save_checkpoint_freq: !!float 1e3


Run model, results will be in ```/content/SRFlow/results```.

In [ ]:
#@title apply model
%cd /content/Colab-SRFlow/code
# SRFlow_DF2K_4X
#!CUDA_VISIBLE_DEVICES=-1 python test.py ./confs/test.yml # CPU
!python test.py ./confs/test.yml

In [ ]:
#@title delete output
%cd /content/
!sudo rm -rf /content/SRFlow/results/test